In [ ]:
import datetime
import os

StatementMeta(, , , Waiting, )

In [ ]:
%pip install openai --upgrade

StatementMeta(, , , Waiting, )


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


### Use defined get_completion() function

In [ ]:
%run ./openai-helper

StatementMeta(, , , Waiting, )

### Create an Anomaly Graph

In [ ]:
## Define kustoAnomlyDataset

##  This query plots a series of data in an anomaly pattern for the number of parking events
kustoQuery = """
let start_of_day = startofday(now());
let end_of_day = start_of_day + 1d; 

parkingsessions
| where start_time >= start_of_day and start_time < end_of_day
| summarize IngestionCount = count() by bin(start_time, 1h)
| make-series num=avg(IngestionCount) default=0 on start_time from start_of_day to end_of_day step 1h
| extend (anomalies, score, baseline) = series_decompose_anomalies(num, 1.5, -1, 'linefit')
"""

StatementMeta(, , , Waiting, )

### Execute Query Against DB

In [ ]:
# The Kusto cluster uri to read the data. The query Uri is of the form https://<>.kusto.data.microsoft.com
kustoUri = "https://trd-7axdk3c8jb09vq4sh0.z3.kusto.fabric.microsoft.com"
# The database to read the data
database = "kql-aihack"
# The access credentials for the write
accessToken = mssparkutils.credentials.getToken(kustoUri)
kustoDf  = spark.read\
            .format("com.microsoft.kusto.spark.synapse.datasource")\
            .option("accessToken", accessToken)\
            .option("kustoCluster", kustoUri)\
            .option("kustoDatabase", database) \
            .option("kustoQuery", kustoQuery).load()
            
 
resultList = [row.asDict() for row in kustoDf.collect()]

StatementMeta(, , , Waiting, )

### Get Analysis

In [ ]:
result = get_completion(f"""

Analyse the following data - return to me a long description of the trend of car parking instances in the format of this. 
Try to be verbose and pick out all key bits of information

-- Format --

Overview:

Points to Note:
 - Very Busy at.... 
 - Odd Peaks here....

-- Data --

{str(resultList)}

""")

print(result)

StatementMeta(, , , Waiting, )

Overview:
The data provided shows the number of car parking instances over a 24-hour period on March 1, 2024. The number of car parking instances ranges from 0 to 22 per hour. There are anomalies noted in the data, but they seem to be minimal. The data also includes a score and baseline for comparison.

Points to Note:
- The car park seems to be very busy in the early morning hours, with a peak of 22 car parking instances at 2:00 AM.
- There is an odd peak at 8:00 AM where the number of instances suddenly jumps from 17 to 20 but drops back down in the following hours.
- The data also shows a significant decrease in car parking instances during the afternoon hours, with a low of 1 instance at 12:00 PM.

Overall, the trend of car parking instances on March 1, 2024, shows fluctuations throughout the day, with distinct peaks and drops in the number of instances. The data suggests that the car park experiences high traffic in the early morning hours and lower traffic during the afternoon. T

Overview:
The data provided shows the number of car parking instances over a 24-hour period on March 1, 2024. The number of car parking instances ranges from 0 to 22 per hour. There are anomalies noted in the data, but they seem to be minimal. The data also includes a score and baseline for comparison.

Points to Note:
- The car park seems to be very busy in the early morning hours, with a peak of 22 car parking instances at 2:00 AM.
- There is an odd peak at 8:00 AM where the number of instances suddenly jumps from 17 to 20 but drops back down in the following hours.
- The data also shows a significant decrease in car parking instances during the afternoon hours, with a low of 1 instance at 12:00 PM.

Overall, the trend of car parking instances on March 1, 2024, shows fluctuations throughout the day, with distinct peaks and drops in the number of instances. The data suggests that the car park experiences high traffic in the early morning hours and lower traffic during the afternoon. T

### Save Analysis in Onelake

In [ ]:
# Now Save the Data
path = '/lakehouse/default/Files'

# Get current year, month, day
now = datetime.datetime.now()
year, month, day = now.strftime("%Y"), now.strftime("%m"), now.strftime("%d")

# Create the directory path
directory_path = f"{path}/{year}/{month}/{day}"
file_path = os.path.join(directory_path, "summary.txt")

# Ensure the directory exists
os.makedirs(directory_path, exist_ok=True)

# Write the data to file
with open(file_path, 'w') as file:
    file.write(result) 

StatementMeta(, , , Waiting, )

### Send Teams Notification

In [16]:
send_teams_message(result)

StatementMeta(, ddce7819-7e0d-4941-acf4-2ecf7a6647de, 27, Finished, Available)